# Introduction to Natural Language Processing 2 Lab04
Authors:
- Jonathan Poelger
- Ethan Machavoine
- Aurelien Rouxel

In [ ]:
#!pip install bertopic
#!pip install -U accelerate
#!pip install evaluate

In [2]:
import numpy as np
from datasets import load_dataset
from bertopic import BERTopic
from umap import UMAP
from transformers import AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding
import evaluate
from transformers import AutoModelForSequenceClassification

# Introduction

In [ ]:
#Loading both of the possible datasets
hate = load_dataset("tweet_eval", "hate")
offensive = load_dataset("tweet_eval", "offensive")

In [4]:
#Exemples of 'hate' dataset
i = 0
printed = 0
while printed < 10:
    if hate["train"]["label"][i] == 1 :
        print(hate["train"]["text"][i], "\n")
        printed += 1
    i+=1


A woman who you fucked multiple times saying yo dick small is a compliment you know u hit that spot 😎 

@user @user real talk do you have eyes or were they gouged out by a rapefugee? 

your girlfriend lookin at me like a groupie in this bitch! 

I AM NOT GOING AFTER YOUR EX BF YOU LIEING SACK OF SHIT ! I'm done with you dude that's why I dumped your ass cause your a lieing 😂😡 bitch 

Send home migrants not in need of protection, Peter Dutton tells UN, HEY DUTTON HOW ABOUT THE ONES THAT HAVE STAYED AND NOT LEFT THE COUNTRY WHEN THEY SHOULD OVERSTAYERS ? WHY DONT YOU GO AND ROUND ALL THEM UP ? 

Cory Booker and Kamala Harris competing for Most Hysterical Woman at the Kavanaugh hearings, Coulter hilariously tweeted.And yes, liberals immediately got triggered on Twitter, saying her joke was offensive. To them we say, suck it up, snowflakes. 

Should have let the Libyan Manchester bomber rot in his own country. How many more mass murderers did the Royal Navy import to Britain? How many more

In [5]:
#Exemples of 'offensive' dataset
i = 0
printed = 0
while printed < 10:
    if offensive["train"]["label"][i] == 1 :
        print(offensive["train"]["text"][i], "\n")
        printed += 1
    i+=1

@user Eight years the republicans denied obama’s picks. Breitbarters outrage is as phony as their fake president. 

@user She has become a parody unto herself? She has certainly taken some heat for being such an....well idiot. Could be optic too  Who know with Liberals  They're all optics.  No substance 

@user Your looking more like a plant #maga #walkaway 

@user Antifa would burn a Conservatives house down and CNN would be there lighting the torches &amp; throwing gas on the flames. 

@user They cite Jones being banned for violating Twitter's ToS. There are blue checkmarks spewing the same, if not worse, kind of shit. If you are going to play the anyone can get banned"" card. Shouldn't these people also receive bans and suspensions? #VerifiedHate"" 

@user also shitbiscuit stole most of the Tempe girls u can't blame that on me.. 

@user @user Well, see, if I start talking at Dana about sensible gun control laws and she ignores me (or calls me a tragedy dry-humping whore"" as she did

## Dataset

After analizing the two datasets, it seems that the 'hate' dataset in more about insults and defamation, while 'offensive' is way more political and about people arguing.\
They also both have a similar amount of exemples (12970 and 14100), so the matter of amount of data isn't relevant to choose.\
Something relevant is the legislation of these datasets. The 'hate' dataset is unde the Creative Commons bCC-BY-NC-4.0 license, which makes it, among other things, not usable for commercial purposes.\
The 'offensive' dataset isn't under any other legislations than the twitter's terms of services, so we can use it for commercial purposes.\

We will then choose the 'offensive' dataset.

# Evaluating the dataset

In [6]:
#Class repartition in the dataset splits
print("train:", round(offensive["train"]["label"].count(0) / len(offensive["train"]["label"]) * 100, 2), "% of 'offensive' tweets")
print("val:", round(offensive["validation"]["label"].count(0) / len(offensive["validation"]["label"]) * 100, 2), "% of 'offensive' tweets")
print("test:", round(offensive["test"]["label"].count(0) / len(offensive["test"]["label"]) * 100, 2), "% of 'offensive' tweets")

train: 66.93 % of 'offensive' tweets
val: 65.33 % of 'offensive' tweets
test: 72.09 % of 'offensive' tweets


- train: 66.93 % of 'offensive' tweets
- val  : 65.33 % of 'offensive' tweets
- test : 72.09 % of 'offensive' tweets

### Getting the topics

In [7]:
random_seed = 42
"""
Given a BERTopic umap model and a dataframe,
computes and prints the main topics
"""
def get_topics(umap_model, df):
    docs = df["train"]["text"] + df["validation"]["text"] + df["test"]["text"]
    topic_model = BERTopic(umap_model=umap_model)
    topics, probs = topic_model.fit_transform(docs)
    ret = topic_model.get_topic_info()[0:5]["Representation"].tolist()
    for i in ret:
        print(i)

umap_model = UMAP(n_neighbors=15, n_components=5, 
                  min_dist=0.0, metric='cosine', random_state=42)


In [8]:
#Topics of the wole dataset
get_topics(umap_model, offensive)

['to', 'the', 'is', 'and', 'you', 'of', 'he', 'she', 'are', 'user']
['right', 'user', 'wrong', 'you', 'correct', 'are', 'absolutely', 'it', 'what', 'should']
['liberals', 'user', 'they', 'are', 'all', 'their', 'liberal', 'that', 'dont', 'crazy']
['brexit', 'tories', 'labour', 'eu', 'housing', 'conservatives', 'uk', 'tory', 'the', 'party']
['nfl', 'football', 'he', 'game', 'team', 'league', 'is', 'his', 'him', 'season']


In [9]:
#Topics of the flagged offensive tweets
get_topics(umap_model, offensive.filter(lambda example: example['label'] == 1))

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

['user', 'is', 'she', 'to', 'and', 'the', 'he', 'you', 'of', 'are']
['gun', 'control', 'the', 'laws', 'guns', 'to', 'user', 'in', 'of', 'that']
['antifa', 'user', 'the', 'fascist', 'to', 'and', 'of', 'they', 'are', 'in']
['maga', 'trump', 'the', 'walkaway', 'user', 'qanon', 'wwg1wga', 'and', 'to', 'for']
['liberals', 'are', 'user', 'all', 'they', 'liberal', 'the', 'of', 'and', 'to']


In [10]:
#topics of the flagged neutral tweets
get_topics(umap_model, offensive.filter(lambda example: example['label'] == 0))

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

['the', 'and', 'to', 'you', 'is', 'of', 'he', 'are', 'in', 'for']
['right', 'user', 'you', 'are', 'correct', 'what', 'wrong', 'it', 'he', 'absolutely']
['liberals', 'user', 'they', 'liberal', 'the', 'all', 'have', 'dont', 'their', 'to']
['kavanaugh', 'maga', 'judge', 'kavanaughs', 'ford', 'testify', 'to', 'feinstein', 'her', 'fbi']
['brexit', 'labour', 'conservatives', 'tories', 'housing', 'eu', 'the', 'uk', 'of', 'tory']


# Topics results

### Most notable topics on the whole dataset :
- right, wrong, correct, should: people arguing
- liberals, conservatives, brexit, tories, eu, uk: politics
- crazy: most likely insulting, but can be used for emphasis (cf. "it's crazy expensive")
- nfl, football, team, league, game, season: sports

### Most notable topics on the flagged offensive subset :
- gun, control, law: legislation around guns
- antifa, facsist: political extremes
- maga, trump, wwg1wga: Donald Trump and his political movement
- quanon:  American political conspiracy theory and political movement
- liberals: politics

### Most notable topics on the not flagged offensive subset :
- right, wrong, correct, should: people arguing
- liberals, tory, brexit, eu, uk: UK politics
- maga, judge, kavanaught, feinstein: american politics
- labour, housing: people common issues 
- fbi: american l federal law enforcement agency (widely used as a joke on social medias)

As we can see, the flagged 'offensive' tweets are mostely about american politics (gun control, Trump, etc.), while the not offensives ones are also about UK, and people arguing about various issues. The 'offensive' subset is also about more sensitive contents and extremes. 

This could create a bias in the model towards american-related arguments and also politics in general for exemple. The exemples later will show these "political", "US" ans "UK" biases at work.

# Evaluate a model

In [ ]:
task='offensive'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

"""
Given a trainer containing a model and a dataset,
Computes and prints the metrics of the model
"""
def compute_metrics(trainer, dataset):
    predictions = trainer.predict(dataset["test"])
    true_labels = dataset["test"]["label"]
    predicted_labels = np.argmax(predictions.predictions, axis=1)
    accuracy = np.mean(predicted_labels == dataset["test"]["label"])
    
    predicted_labels = list(predicted_labels)
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    
    for i in range(len(true_labels)):
        true_positives += predicted_labels[i] == 1 and true_labels[i] == 1
        false_positives += predicted_labels[i] == 1 and true_labels[i] == 0
        false_negatives += predicted_labels[i] == 0 and true_labels[i] == 1
    
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1_score = 2 * (precision * recall) / (precision + recall)
    
    print("Accuracy :", accuracy)
    print("Precision:", precision)
    print("Recall   :", recall)
    print("F1-score :", f1_score)
    return trainer, dataset

"""
Given a model name,
Creates a tokenizer and returns the tokenized dataset, tokenizer and model name
"""
def load_data(checkpoint):
    raw_datasets = load_dataset("tweet_eval", "offensive")
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    tokenized_datasets = raw_datasets.map(lambda example: tokenizer.__call__(example["text"], truncation=True), batched=True)
    return tokenized_datasets, tokenizer, checkpoint

"""
Given a tokenized dataset, tokenizer and a model name,
Creates a trainer and returns it as well as the dataset
"""
def create_model(tokenized_datasets, tokenizer, checkpoint ,num_train_epochs=1):
    training_args = TrainingArguments("test-trainer", num_train_epochs=num_train_epochs, report_to="none")
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
    trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator= DataCollatorWithPadding(tokenizer=tokenizer),
    tokenizer=tokenizer,
    )
    return trainer, tokenized_datasets

trainer, dataset = compute_metrics(*create_model(*load_data(MODEL)))

Accuracy : 0.8593023255813953\
Precision: 0.7960199004975125\
Recall   : 0.6666666666666666\
F1-score : 0.7256235827664399

### Prediction failures analysis

In [12]:
#Get all the needed vectors of prediction and ground truth
preds = trainer.predict(dataset["test"]).predictions
truth = dataset["test"]["label"]
texts =  dataset["test"]["text"]
labels = np.argmax(preds, axis=1)

tp = []
tn = []
fp = []
fn = []
#Classify the predictions using confusion matrix 
for i in range(len(preds)):
        if truth[i] == 0 and labels[i] == 0:
            tn.append((texts[i], max(preds[i])))
        elif truth[i] == 0 and labels[i] == 1:
            fp.append((texts[i], max(preds[i])))
        elif truth[i] == 1 and labels[i] == 0:
            fn.append((texts[i], max(preds[i])))
        elif truth[i] == 1 and labels[i] == 1:
            tp.append((texts[i], max(preds[i])))

def takeSecond(elem):
    return elem[1]
tp.sort(key=takeSecond, reverse=True)
fp.sort(key=takeSecond, reverse=True)
tn.sort(key=takeSecond, reverse=True)
fn.sort(key=takeSecond, reverse=True)

#Outputs the most wrong awnsers
print("\tfalse positive")
for i in range(5):
    print(fp[i][0])

print("\n\tfalse negative")
for i in range(5):
    print(fn[i][0])

	false positive
@user I guess that’s where swamp ass originated
Are you fucking serious?
An American Tail really is one of the most underrated animations ever ever ever. Fuck I cried in this scene
@user @user Bull crap. You know she doesn't care.  She is trying to get attention for her Presidential run.  Do you see any other Senator giving nonsense?  Nope.
#Room25 is actually incredible, Noname is the shit, always has been,  and I’m seein her in like 5 days in Melbourne. Life is good. Have a nice day.

	false negative
#Liberals / #Democrats THIS is what you stand for. If not, then #WalkAway
#Liberals Are Reaching Peak Desperation To Call On #PhillipRuddock To Talk With #Turnbull To Convince Him To Help with #WentworthVotes 18 Sept 2018  @user #Auspol #LNP #NSWpol @user  @user @user #LNPMemes
#BREXIT deal HAS been reached - and will be unveiled at special summit in NOVEMBER, Has @user sold out the #UK to the eu??? She better have not or the @user are finished!! @user
#NoPasaran: Unity d

### False positive
@user I guess that’s where swamp ass originated


Are you fucking serious?


An American Tail really is one of the most underrated animations ever ever ever. Fuck I cried in this scene


@user @user Bull crap. You know she doesn't care.  She is trying to get attention for her Presidential run. Do you see any other Senator giving nonsense?  Nope.


\#Room25 is actually incredible, Noname is the shit, always has been, and I’m seein her in like 5 days in Melbourne. Life is good. Have a nice day.

### Fasle negative
\#Liberals / #Democrats THIS is what you stand for. If not, then #WalkAway


\#Liberals Are Reaching Peak Desperation To Call On #PhillipRuddock To Talk With #Turnbull To Convince Him To Help with #WentworthVotes 18 Sept 2018  @user #Auspol #LNP #NSWpol @user  @user @user #LNPMemes


\#BREXIT deal HAS been reached - and will be unveiled at special summit in NOVEMBER, Has @user sold out the #UK to the eu??? She better have not or the @user are finished!! @user


\#NoPasaran: Unity demo to oppose the far-right in #London – #antifa #Oct13 — Enough is Enough!


\#America  ... tear down that #Wall! #tcot #partisanship #Trump #thewall #Borderwall #liberty #civilsociety #think #Conservatives #Democrats #Progressives #liberals #Independent #libertarians #GOP #DNC #CriticalThinking


### Possible issues and biases
The 5 most wrong awsners in the false positives all have a vulgar word within them, but that's the only common factor that could have them wrongly calssified. The "US bias" can have an impact on the comment about "American Tail" (a movie title), and the "political bias" can have an impact on the comment with the words 'Presidantial' and 'Senator'


The 5 most wrong awsners in the false negatives all have a lot of hashtags in them, some of them even in the middle of a sentense. The model may take them as separate words and so not link them with their context like it usually does. Also all of them are about UK politics, so the "UK bias" play a role in the classification


In [13]:

"""
This code aims to load english tweets from the twitter database.
Unfortunately, while loading them isn't that much of an issue,
I couldn't find a way to properly pass them to the network.
"""
import json

tweets = []
for i in range(30,60):
    with open('/kaggle/input/tweets/'+ str(i) +'.json') as f:
        for jsonObj in f:
            tweetDict = json.loads(jsonObj)
            if "lang" in tweetDict and tweetDict["lang"] == 'en':
                tweets.append(tweetDict['text'])



### Advantages of using a pre-trained transformer:

- Better generalization and performance compared to naive Bayes, which relies on simple probabilities.

- Transformers learns the relationships between words in a sentence, and context. This way, it can understand more complex language structures compared to Naive Bayes that takes each words one by one. It can also see dependencies between words very far away in the texts, and is in fact very good at it.

- Pre-trained transformers can be fine-tuned with a few labeled data. This makes them more effective in scenarios where labeled data isn't easy to find. Naive Bayes requires a larger labeled dataset for training. It also gives it the ability to be quickly trained for new domains or problems.

- Pre-trained transformers like BERT or GPT have shown very good results on natural language processing tasks. Naive Bayes have less good results in basically every domains. In fact, while Naive Bayes is limited to classification, Transformers can handle a wide range of problems

- Pre-trained transformer models, with libraries like Hugging Face Transformers, are already easily available for wide range of problems. This accessibility and fast development make it generally a better choice over Naive Bayes which is harder to find and maintain.

### Annotate data

In [14]:
def get_tweets():
    pass

### Labeling guidelines

1. Offensive Class Definition:

   The offensive class in this annotation guideline are tweets that contain offensive content. Offensive content is targeting directs a person, a group of people, an institution, or an idea and spreading misinformations, insults, hate speech, threats, or any other kind of objectively false and/or harmful information.
   
   The subject ranges from personal attributes to groups, such as race, gender, religion, ethnicity, sexual orientation, political view, appearance, personal preferences and opinions, etc.
   
   A negative comment isn't necessarily offensive. Information that is objectively and unmistakably true cannot be flagged as offensive, even if the recipient doesn't like what is said. More on that subject on the "Ambiguous" part.
   
   A comment that clearly isn't offensive will be treated as "Neutral" here. There will also be a "Not annotable" class

2. Offense Examples:
   - _"@user lemme tell ya bro, arab people are fucking terrorists"_ => objectively false and targetting a group of person, spreading false and harmful informations towards them
   - _"@user You so fat you have your own gravitational field"_ => absolutely impossible and shaming a person's appearance
   - _"I can't belive #wwg1wgaat is again promoting school shootings for the sake of it..... #guncontrol #Trump"_ => misinformation about a political party 

3. Negative but not offensive comments (counterpart of above section):
    - _"The latest attack has been revamped by Islamist extermists"_
    - _"@user, your extrem overweight is bad for your health and can causes multiple vascular diseases"_
    
    => By stating only stating facts, not confusing words and ideas and being polite, The disscutions can be kept civil about any subjects and thus are not offensive.
    
    - _"#wwg1wga is promoting gun rights in America while there is still school shootings on a regular basis. This is very illogical in my opinion and I think they souldn't do that for the sake of our childen's safety."_
    
    => One can voice their opinion and disagreeing without it being offensive. As above, stating facts and staying polite is a good way to state disagreement with an idea without it being offensive, even on serious and sensitive matters.
    
4. Ambiguous Cases:
   It is important to pay attention to ambiguous cases where the presence of offense may not be immediately evident. Subtle forms of offensive language, sarcasm, satire, or irony can be hard to juge as offensive or not. The annotators should consider the overall context, tone, and intent of the tweet when making judgments.

5. Can't Tell / Not Annotable Class:
   When an exemple is too ambibuous, we can use the "Can't tell / not annotable" class. It is usefull as the annotator can be perplexed as how to classify borderline exemples.
   
6. Consistency:
   When considering different languages, cultures, and regional contexts, some exemples may be hard to classify. Offensive content can vary across languages and cultures, so annotators should be aware of potential cultural biases and understand the context of the specific language and culture being analyzed. For the sake of impartiality we need discussion among annotators, as well as multiple annotators labeling the same exemples. If disagreement on a label arises, deeper rescearch and disscutions will be made to ensure consistency in labeling.

In [16]:
"""
Given a list of tweets and possibly a number of target class to get,
Provides with and easy way to annotate the data
"""
def annotate(tweets, target_class = 50):
    dataset = []
    offensive = 0
    for tweet in tweets:
        print(tweet)
        label = input()
        dataset.append((tweet,label))
        if label == 1:
            offensive +=1
            if offensive == target_class:
                break
    return dataset